In [1]:
from sklearn.datasets import load_breast_cancer
import pandas as pd
import seaborn as sns

import pycarrot as pc

%load_ext autoreload
%autoreload 2

pd.options.display.max_columns = 2000

sns.set_theme()

/home/george/Desktop/pycarrot/.venv/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# bc = load_breast_cancer()
# X = pd.DataFrame(bc.data, columns=bc.feature_names)
# y = pd.Series(bc.target, name="class")
# df = pd.concat([X, y], axis=1)
# df.head()

In [2]:
df = pd.read_csv(
    "./data/breast_cancer_cat/breast-cancer.data",
    names=[
        "class",
        "age",
        "menopause",
        "tumor-size",
        "inv-nodes",
        "node-caps",
        "deg-malig",
        "breast",
        "breast-quad",
        "irradiat",
    ],
)
df.head()

,class,age,menopause,tumor-size,inv-nodes,node-caps,deg-malig,breast,breast-quad,irradiat
0,no-recurrence-events,30-39,premeno,30-34,0-2,no,3,left,left_low,no
1,no-recurrence-events,40-49,premeno,20-24,0-2,no,2,right,right_up,no
2,no-recurrence-events,40-49,premeno,20-24,0-2,no,2,left,left_low,no
3,no-recurrence-events,60-69,ge40,15-19,0-2,no,2,right,left_up,no
4,no-recurrence-events,40-49,premeno,0-4,0-2,no,2,right,right_low,no


In [3]:
# pc.feat_analysis.get_distribution(df)

In [4]:
# sns.jointplot(data=df, x="deg-malig", y="breast", hue="class")

## Training

In [5]:
df.columns

Index(['class', 'age', 'menopause', 'tumor-size', 'inv-nodes', 'node-caps',
       'deg-malig', 'breast', 'breast-quad', 'irradiat'],
      dtype='object')

In [32]:
config = pc.init_config("./config_bc_cat.yml")
config

{'modelling': {'target_clf': 'class',
  'numeric_cols': ['deg-malig'],
  'categorical_cols': ['age',
   'menopause',
   'tumor-size',
   'inv-nodes',
   'node-caps',
   'breast',
   'breast-quad',
   'irradiat'],
  'normalization': False}}

In [33]:
setup, X_sample, y_sample = pc.modelling.prepare_data(
    train_data=df,
    config=config,
)

In [8]:
X_sample

,deg-malig,age_20-29,age_30-39,age_40-49,age_50-59,age_60-69,age_70-79,menopause_ge40,menopause_lt40,menopause_premeno,tumor-size_0-4,tumor-size_10-14,tumor-size_15-19,tumor-size_20-24,tumor-size_25-29,tumor-size_30-34,tumor-size_35-39,tumor-size_40-44,tumor-size_45-49,tumor-size_5-9,tumor-size_50-54,inv-nodes_0-2,inv-nodes_12-14,inv-nodes_15-17,inv-nodes_24-26,inv-nodes_3-5,inv-nodes_6-8,inv-nodes_9-11,node-caps_?,node-caps_no,node-caps_yes,breast_left,breast_right,breast-quad_?,breast-quad_central,breast-quad_left_low,breast-quad_left_up,breast-quad_right_low,breast-quad_right_up,irradiat_no,irradiat_yes
0,3,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,1,0
1,2,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,1,0
2,2,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,1,0
3,2,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0
4,2,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,1,0


In [10]:
compare_df, algo_list, model_list = pc.modelling.compare_algorithms(
    setup=setup,
    include=[
        "lr",
        "dt",
        "rf",
        "ridge",
        "perceptron",
        "passive-aggressive",
        "extratree",
        "extratrees",
        "knn",
        "nb",
        "linearsvc",
        "rbfsvc",
    ],
    sort="f1",
    return_models=True,
)
compare_df

,algorithm,accuracy,precision,recall,f1,roc_auc,Fit time (s)
0,nb,0.549,0.447,0.835,0.552,0.659,0.105
1,linearsvc,0.671,0.575,0.365,0.392,0.605,0.104
2,lr,0.685,0.581,0.376,0.382,0.634,0.268
3,ridge,0.668,0.549,0.353,0.375,0.609,0.123
4,extratree,0.636,0.387,0.376,0.358,0.561,0.091
5,rf,0.671,0.403,0.353,0.348,0.649,2.813
6,extratrees,0.661,0.357,0.365,0.342,0.613,2.410
7,knn,0.717,0.534,0.271,0.335,0.621,0.070
8,dt,0.633,0.351,0.329,0.330,0.550,0.065
9,rbfsvc,0.703,0.569,0.306,0.325,0.676,0.111


In [11]:
algo_list

['nb',
 'linearsvc',
 'lr',
 'ridge',
 'extratree',
 'rf',
 'extratrees',
 'knn',
 'dt',
 'rbfsvc',
 'passive-aggressive',
 'perceptron']

In [12]:
model_list

[GaussianNB(),
 LinearSVC(),
 LogisticRegression(),
 RidgeClassifier(),
 ExtraTreeClassifier(),
 RandomForestClassifier(),
 ExtraTreesClassifier(),
 KNeighborsClassifier(),
 DecisionTreeClassifier(),
 SVC(),
 PassiveAggressiveClassifier(),
 Perceptron()]

In [ ]:
def optimize_after_compare_algorithms():
    algorithm, reference_metric = compare_df.loc[
        compare_df["algorithm"] == algo_list[0], ["algorithm", "f1"]
    ].values[0]
    best_feature_list = pc.modelling.reduce_feature_space(
        setup, algorithm, "f1", reference_metric, acceptable_loss=0.5
    )
    return best_feature_list
    
# optimize_after_compare_algorithms()

In [13]:
le = setup.y_clf_encoder
preds = model_list[-1].predict(X_sample)
le.inverse_transform(preds)

array(['recurrence-events', 'recurrence-events', 'recurrence-events',
       'recurrence-events', 'no-recurrence-events'], dtype=object)

In [14]:
print(*[(i, class_) for i, class_ in enumerate(le.classes_)])

(0, 'no-recurrence-events') (1, 'recurrence-events')


In [44]:
compare_df, model_list = (
        pc.modelling.tune_hyperparams(
               setup=setup,
               include=["lr", "nb"],
               optimize="f1",
               n_trials=50,
               return_models=True,
        )
)
compare_df

[I 2022-07-13 17:33:43,665] A new study created in memory with name: study_lr
[I 2022-07-13 17:33:43,782] Trial 0 finished with value: 0.0 and parameters: {'C': 0.00022296039863750085, 'l1_ratio': 0.9081588242341383}. Best is trial 0 with value: 0.0.


The algorithms ['lr'] work suboptimally without normalized data. Consider turning it on within the config.


[I 2022-07-13 17:33:43,910] Trial 1 finished with value: 0.0 and parameters: {'C': 0.0076643202071359105, 'l1_ratio': 0.8632711987883429}. Best is trial 0 with value: 0.0.
/home/george/Desktop/pycarrot/.venv/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
[I 2022-07-13 17:33:45,309] Trial 2 finished with value: 0.36870274652883345 and parameters: {'C': 28.70933015266027, 'l1_ratio': 0.43333346155821373}. Best is trial 2 with value: 0.36870274652883345.
[I 2022-07-13 17:33:45,536] Trial 3 finished with value: 0.0 and parameters: {'C': 0.022584349507672293, 'l1_ratio': 0.8703849785205707}. Best is trial 2 with value: 0.36870274652883345.
[I 2022-07-13 17:33:46,597] Trial 4 finished with value: 0.39802516324255455 and parameters: {'C': 5.267500586275035, 'l1_ratio': 0.20843522900988143}. Best is trial 4 with value: 0.39802516324255455.
[I 2022-07-13 17:33:46,854] Trial 5 fini

,algorithm,metric,hyperparams
0,nb,0.577910,"{'priors': None, 'var_smoothing': 7.5566077239..."
1,lr,0.398025,"{'C': 5.267500586275035, 'class_weight': None,..."


In [49]:
compare_df

,algorithm,metric,hyperparams
0,nb,0.577910,"{'priors': None, 'var_smoothing': 7.5566077239..."
1,lr,0.398025,"{'C': 5.267500586275035, 'class_weight': None,..."


In [46]:
compare_df.iloc[0]["hyperparams"]

{'priors': None, 'var_smoothing': 7.5566077239176915e-06}

In [50]:
best_feature_list = pc.modelling.reduce_feature_space(
    setup=setup,
    algorithm= "nb",metric="f1",
    reference_metric=compare_df.iloc[0]["metric"],
    acceptable_loss=0.5,
    hyperparams=compare_df.iloc[0]["hyperparams"]
)
best_feature_list

New metric: 0.589, worst feature: breast-quad_right_up
Updating reference metric...
New metric: 0.595, worst feature: breast_left
Updating reference metric...
New metric: 0.598, worst feature: tumor-size_20-24
Updating reference metric...
New metric: 0.598, worst feature: age_40-49
New metric: 0.6, worst feature: breast-quad_right_low
Updating reference metric...
New metric: 0.6, worst feature: tumor-size_25-29
New metric: 0.602, worst feature: age_60-69
Updating reference metric...
New metric: 0.613, worst feature: breast_right
Updating reference metric...
New metric: 0.613, worst feature: tumor-size_15-19
New metric: 0.62, worst feature: tumor-size_50-54
Updating reference metric...
New metric: 0.621, worst feature: menopause_lt40
Updating reference metric...
New metric: 0.623, worst feature: breast-quad_central
Updating reference metric...
New metric: 0.625, worst feature: node-caps_?
Updating reference metric...
New metric: 0.625, worst feature: menopause_ge40
New metric: 0.628, wo

['deg-malig',
 'age_20-29',
 'age_30-39',
 'age_50-59',
 'age_70-79',
 'menopause_premeno',
 'tumor-size_0-4',
 'tumor-size_10-14',
 'tumor-size_30-34',
 'tumor-size_35-39',
 'tumor-size_45-49',
 'tumor-size_5-9',
 'inv-nodes_0-2',
 'inv-nodes_12-14',
 'inv-nodes_15-17',
 'inv-nodes_24-26',
 'inv-nodes_3-5',
 'inv-nodes_6-8',
 'inv-nodes_9-11',
 'node-caps_yes',
 'breast-quad_?',
 'breast-quad_left_low',
 'breast-quad_left_up',
 'irradiat_no',
 'irradiat_yes']

In [47]:
def optimize_after_tuning():
    best_feature_list = pc.modelling.reduce_feature_space(
        setup=setup,
        algorithm= "nb",metric="f1",
        reference_metric=compare_df.iloc[0]["metric"],
        acceptable_loss=0.5,
        hyperparams=compare_df.iloc[0]["hyperparams"]
    )
    return best_feature_list
    
# optimize_after_tuning()

GaussianNB(var_smoothing=7.5566077239176915e-06)